### Linear Model Selection Example 2.1
For the  **Credit** example, we begin with the so-called *null model* $ \mathcal{M}_{0} $, which contains no predictors:
\begin{equation*}
balance
=\beta_{0}+\epsilon
\end{equation*}
Then, we add a predictor variable to the null model. 
For this example, we will write two **Python**-functions, one to fit a linear model and return important scoring metrics and one to add one predictor to a model, based on the returned scores. 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load data
df = pd.read_csv('./data/Credit.csv')

# Convert Categorical variables
df = pd.get_dummies(data=df, drop_first=True, 
                    prefix=('Gender_', 'Student_', 
                            'Married_', 'Ethnicity_'))

x_full = df.drop(columns='Balance')
y = df['Balance']


def fit_linear_reg(x, y):
    '''Fit Linear model with predictors x on y 
    return AIC, BIC, R2 and R2 adjusted '''
    x = sm.add_constant(x)
    # Create and fit model
    model_k = sm.OLS(y, x).fit()
    
    # Find scores
    BIC = model_k.bic
    AIC = model_k.aic
    R2 = model_k.rsquared
    R2_adj = model_k.rsquared_adj
    RSS = model_k.ssr
    
    # Return result in Series
    results = pd.Series(data={'BIC': BIC, 'AIC': AIC, 'R2': R2,
                              'R2_adj': R2_adj, 'RSS': RSS})
    
    return results


def add_one(x_full, x, y, scoreby='RSS'):
    ''' Add possible predictors from x_full to x, 
    Fit a linear model on y using fit_linear_reg
    Returns Dataframe showing scores as well as best model '''
    # Predefine DataFrame
    x_labels = x_full.columns
    zeros = np.zeros(len(x_labels))
    results = pd.DataFrame(
        data={'Predictor': x_labels.values, 'BIC': zeros, 
               'AIC': zeros, 'R2': zeros, 
               'R2_adj': zeros, 'RSS': zeros})

    # For every predictor find R^2, RSS, and AIC
    for i in range(len(x_labels)):
        x_i = np.concatenate((x, [np.array(x_full[x_labels[i]])]))
        results.iloc[i, 1:] = fit_linear_reg(x_i.T, y)
        
    # Depending on where we scoreby, we select the highest or lowest
    if scoreby in ['RSS', 'AIC', 'BIC']:
        best = x_labels[results[scoreby].argmin()]
    elif scoreby in ['R2', 'R2_adj']:
        best = x_labels[results[scoreby].argmax()]
        
    return results, best 


# Define the empty predictor
x_empty = [np.zeros(len(y))]

results, best1 = add_one(x_full, x_empty, y)

print(results[['Predictor', 'AIC', 'R2', 'RSS']], 
      '\n\nBest predictor is:',  best1)

               Predictor          AIC        R2           RSS
0             Unnamed: 0  6042.696603  0.000037  8.433681e+07
1                 Income  5945.894250  0.214977  6.620874e+07
2                  Limit  5499.982633  0.742522  2.171566e+07
3                 Rating  5494.781548  0.745848  2.143512e+07
4                  Cards  6039.710202  0.007475  8.370950e+07
5                    Age  6042.709965  0.000003  8.433963e+07
6              Education  6042.685316  0.000065  8.433443e+07
7         Gender__Female  6042.526817  0.000461  8.430102e+07
8           Student__Yes  6014.932656  0.067090  7.868154e+07
9           Married__Yes  6042.698437  0.000032  8.433720e+07
10      Ethnicity__Asian  6042.672799  0.000096  8.433179e+07
11  Ethnicity__Caucasian  6042.706987  0.000011  8.433900e+07 

Best predictor is: Rating


We will save the definition used in a helper file, **LMS_def.py**.

We now choose the *best* variable in the sense that adding this variable leads to the regression model with the lowest RSS or the highest $ R^2 $. The variable that results in the model with the lowest RSS is in this case **rating**. 

Thus, we have found the model $ \mathcal{M}_{1} $
\begin{equation*}
balance
=\beta_{0}+\beta_{1}\cdot rating +\epsilon
\end{equation*}


We now add a further predictor variable to this model by first updating the reference model and removing the chosen predictor from the set of possible predictors. Subsequently, we can run the same procedure and decide which predictor variable to add next.

In [2]:
# Update the empty predictor with the best predictor
x_1 = [df[best1]]
# Remove the chosen predictor from the list of options
x_red = x_full.drop(columns=best1, errors='ignore')

results, best2 = add_one(x_red, x_1, y)

print(results[['Predictor', 'AIC', 'R2', 'RSS']], 
      '\n\nBest predictor is:',  best2)

               Predictor          AIC        R2           RSS
0             Unnamed: 0  5496.518489  0.746016  2.142103e+07
1                 Income  5212.557085  0.875118  1.053254e+07
2                  Limit  5496.632982  0.745943  2.142716e+07
3                  Cards  5494.187124  0.747492  2.129654e+07
4                    Age  5484.481339  0.753545  2.078601e+07
5              Education  5496.272851  0.746171  2.140788e+07
6         Gender__Female  5496.481640  0.746039  2.141906e+07
7           Student__Yes  5372.232473  0.813849  1.569996e+07
8           Married__Yes  5494.569548  0.747250  2.131691e+07
9       Ethnicity__Asian  5496.067431  0.746302  2.139689e+07
10  Ethnicity__Caucasian  5496.772749  0.745854  2.143465e+07 

Best predictor is: Income


We select again the variable that leads when added to the reference model to the lowest RSS. In this case, we select the predictor variable **income** which gives us the model $ \mathcal{M}_2 $:
\begin{equation*}
balance
=\beta_{0}+\beta_{1}\cdot rating+\beta_{2}\cdot income +\epsilon
\end{equation*}

This procedure will be repeated. In particular, we will add one variable among the 
remaining $ p - 2 $ variables to the model $ \mathcal{M}_{2} $. The resulting model with the lowest RSS will become model $ \mathcal{M}_{3} $.

In [3]:
# Update the empty predictor with the best predictor
x_2 = np.concatenate((x_1, [df[best2]]))
# Remove the chosen predictor from the list of options
x_red = x_red.drop(columns=best2, errors='ignore')

results, best3 = add_one(x_red, x_2, y)

print(results[['Predictor', 'AIC', 'R2', 'RSS']], 
      '\n\nBest predictor is:',  best3)

              Predictor          AIC        R2           RSS
0            Unnamed: 0  5214.551863  0.875120  1.053240e+07
1                 Limit  5210.950291  0.876239  1.043800e+07
2                 Cards  5214.477534  0.875143  1.053045e+07
3                   Age  5211.113461  0.876188  1.044226e+07
4             Education  5213.765645  0.875365  1.051172e+07
5        Gender__Female  5214.521087  0.875129  1.053159e+07
6          Student__Yes  4849.386992  0.949879  4.227219e+06
7          Married__Yes  5210.930247  0.876245  1.043747e+07
8      Ethnicity__Asian  5212.042074  0.875901  1.046653e+07
9  Ethnicity__Caucasian  5213.976405  0.875299  1.051726e+07 

Best predictor is: Student__Yes


We can automatically select the n-best predictors using SequentialFeatureSelector from **sklearn.feature_selection**. Therfore, we need to define the Linear model with **sklearn.linear_model.LinearRegression**. 
The chosen predictors are returned in the support_ attribute.
**Note**: If *None* features are selected, the algorithm automatically choses half number of features given. 

In this case, the predictor **student** turns out to be the variable that we add to model $ \mathcal{M}_{2} $ to obtain model $ \mathcal{M}_{3} $:
\begin{equation*}
balance
=\beta_{0}+\beta_{1}\cdot rating+\beta_{2}\cdot income +\beta_{3}\cdot student+\epsilon
\end{equation*}
We will end up with the following models: 
 $\mathcal{M}_{0},\mathcal{M}_{1},\ldots,\mathcal{M}_{10}$. 

But how are we going to identify the *best* model among these 11 models? We may base our decision on the value of the AIC which is listed in the **Python** output. This statistic allows us to compare different models with each other. We will later discuss the AIC in greater detail. 

The procedure we have followed can also be automated using **sklear.feature\_selection**. However, this is a relatively new package, which is not as flexible or extensive yet. 

In [4]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# define Linear Regression Model in sklearn
linearmodel = LinearRegression()
# Sequential Feature Selection using sklearn
sfs = SequentialFeatureSelector(linearmodel, n_features_to_select=3, 
                                direction='forward')
sfs.fit(x_full, y)

# Print Chosen variables
print(x_full.columns[sfs.support_].values)


['Income' 'Rating' 'Student__Yes']


The model with three predictor variables includes the variables **income**, **rating** and **student**.